In [ ]:
%env MUJOCO_GL=egl

import os
import mujoco
import mediapy
import numpy as np
import pinocchio as pin
from scipy.spatial.transform import Rotation

from dlabsim import DLABSIM_ASSERT_DIR

np.set_printoptions(precision=5, suppress=True, linewidth=500)

In [ ]:
def get_site_tmat(mj_data, site_name):
    tmat = np.eye(4)
    tmat[:3,:3] = mj_data.site(site_name).xmat.reshape((3,3))
    tmat[:3,3] = mj_data.site(site_name).xpos
    return tmat

def get_camera_tmat(mj_data, cam_name):
    tmat = np.eye(4)
    tmat[:3,:3] = mj_data.cam(cam_name).xmat.reshape(3,3)
    tmat[:3,3] = mj_data.cam(cam_name).xpos
    return tmat

def get_body_tmat(mj_data, body_name):
    tmat = np.eye(4)
    tmat[:3,:3] = Rotation.from_quat(mj_data.body(body_name).xquat[[1,2,3,0]]).as_matrix()
    tmat[:3,3] = mj_data.body(body_name).xpos
    return tmat

In [ ]:
mjcf_file_path = "mjcf/exhibition_conference.xml"
mjcf_file = os.path.join(DLABSIM_ASSERT_DIR, mjcf_file_path)

mj_model = mujoco.MjModel.from_xml_path(mjcf_file)
mj_data = mujoco.MjData(mj_model)

renderer = mujoco.Renderer(mj_model, 480, 640)

options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
options.flags[mujoco.mjtVisFlag.mjVIS_COM] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_PERTFORCE] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_PERTOBJ] = True
# options.frame = mujoco.mjtFrame.mjFRAME_BODY.value
options.frame = mujoco.mjtFrame.mjFRAME_SITE.value

mj_data.qpos[:] = mj_model.key("carry").qpos

mujoco.mj_forward(mj_model, mj_data)

renderer.update_scene(mj_data, -1, options)
rgb = renderer.render()
mediapy.show_image(rgb)

In [ ]:
# rgt_endp  = [1.452 0.876 1.375]
# rgt_ende  = [-2.187  0.047 -0.297]

q = np.array([ 0.752,  0.961,  0.   ,  1.   ,  0.   ,  0.   , -0.029])

quat = q[3:]
tmat = np.eye(4)
tmat[:3,:3] = Rotation.from_quat(quat[[1,2,3,0]]).as_matrix()
tmat[:3,3] = q[:3]
print(tmat)

posi_homo = np.array([1.452, 0.876, 1.375, 1.0])
lft_endp = np.linalg.inv(tmat) @ posi_homo
print(lft_endp)

In [ ]:
0.187 + lft_endp[2]

In [ ]:
name_lst = mj_model.names.decode("utf-8").split("\x00")
for n in name_lst:
    if n.startswith("p1_"):
        print(n)

In [ ]:
tmat_lft_end = get_site_tmat(mj_data, "lft_endpoint")
tmat_rgt_end = get_site_tmat(mj_data, "rgt_endpoint")
print(tmat_lft_end[:3,3])
print(tmat_rgt_end[:3,3])
# tmat_rgt_cam = get_site_tmat(mj_data, "lft_handeye")
# np.linalg.inv(tmat_rgt_end) @ tmat_rgt_cam

In [ ]:
tmat1 = get_site_tmat(mj_data, "top")
tmat2 = get_body_tmat(mj_data, "slide_link")
tmat3 = get_body_tmat(mj_data, "lft_arm_link3")
print(tmat1)
print(tmat2)
print(tmat3)
# (tmat2 - tmat1)[2,3]

In [ ]:
print(mj_data.site("lft_endpoint").xpos)
print(mj_data.site("rgt_endpoint").xpos)

In [ ]:
tmat_chest = get_body_tmat(mj_data, "slide_link")
print(tmat_chest)
foot_print = get_body_tmat(mj_data, "mmk2")
print(foot_print)
footprint2chest = np.linalg.inv(foot_print) @ tmat_chest
print(footprint2chest)
tmat_headcam = get_body_tmat(mj_data, "head_cam")
print(tmat_headcam)

In [ ]:
Rotation.from_matrix(mj_data.geom("assembly_board").xmat.reshape((3,3))).as_quat()[[3,0,1,2]]

In [ ]:
tmat_lft_base = get_body_tmat(mj_data, "lft_arm_base")
print(np.linalg.inv(tmat_chest) @ tmat_lft_base)
tmat_rgt_base = get_body_tmat(mj_data, "rgt_arm_base")
print(np.linalg.inv(tmat_chest) @ tmat_rgt_base)

In [ ]:
tmat_head_cam = get_camera_tmat(mj_data, "head_cam")
print(tmat_head_cam)
tmat_head_pitch = get_body_tmat(mj_data, "head_pitch_link")
print(tmat_head_pitch)
hp2hc = np.linalg.inv(tmat_head_pitch) @ tmat_head_cam
quat = Rotation.from_matrix(hp2hc[:3,:3]).as_quat()[[3,0,1,2]]
print(quat)

In [ ]:
tmat_lft_cam = get_site_tmat(mj_data, "lft_handeye")
tmat_lft_end = get_site_tmat(mj_data, "lft_endpoint")
print(np.linalg.inv(tmat_lft_end) @ tmat_lft_cam)
print(np.linalg.inv(tmat_lft_base) @ tmat_lft_cam)

In [2]:
import cv2

img = cv2.imread("/home/tatp/ws/GreatWall/DLabSim/models/textures/logo/airbot.png")
img = 255 - img
cv2.imwrite("/home/tatp/ws/GreatWall/DLabSim/models/textures/logo/white_airbot.png", img)

cv2.namedWindow("img")
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()